Import Dependencies
-

In [1]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.model_selection import train_test_split
from deepctr.models import DeepFM
from deepctr.feature_column import SparseFeat, DenseFeat,get_feature_names

from sklearn.metrics import log_loss, roc_auc_score
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt

import time
import sys

import tensorflow as tf
from tensorflow.python.keras.models import  save_model,load_model
from tensorflow.keras import layers
from deepctr.layers import custom_objects

import numpy as np
import math
from collections import deque
import random
import heapq

Load Data
-

In [31]:
#Load Raw data
header_row = ['label','I1','I2','I3','I4','I5','I6','I7','I8','I9','I10','I11','I12','I13','C1','C2','C3','C4','C5','C6','C7','C8','C9','C10','C11','C12','C13','C14','C15','C16','C17','C18','C19','C20','C21','C22','C23','C24','C25','C26']
start_load_time = time.time()
data = pd.read_csv('../data/kaggle-display-advertising-challenge-dataset/train.txt', sep='\t', names=header_row, nrows=3000000)
print("Loading Time:",time.time()-start_load_time)

#Enocde missing values
sparse_features = ['C' + str(i) for i in range(1, 27)]
dense_features = ['I'+str(i) for i in range(1, 14)]
data[sparse_features] = data[sparse_features].fillna('-1', )
data[dense_features] = data[dense_features].fillna(0,)
target = ['label']

#Pre-Truncate: Label Encoding for sparse features,and do simple Transformation for dense features
for feat in sparse_features:
    lbe = LabelEncoder()
    data[feat] = lbe.fit_transform(data[feat])

#Remove columns with too many unique elements (limited by embedding vector inversion model)
trunc_sparse_features = []
for col in sparse_features:
    if data[col].nunique() > 55000:
        data = data.drop(columns=[col])
    else:
        trunc_sparse_features.append(col)
        
sparse_features = ['C' + str(i) for i in range(1, len(data.columns) - len(dense_features))]
header_row = ['label']
for feat in dense_features:
    header_row.append(feat)
for feat in sparse_features:
    header_row.append(feat)
    
data.columns = header_row

#Post Truncate: Label Encoding for sparse features,and do simple Transformation for dense features
for feat in sparse_features:
    lbe = LabelEncoder()
    data[feat] = lbe.fit_transform(data[feat])
    
mms = MinMaxScaler(feature_range=(0, 1),copy=False)
#data[dense_features] = mms.fit_transform(data[dense_features]) #Use before model training
mms.fit_transform(data[dense_features]) #train mms on distribution, but keep data values the same for example recommendations to be explained

# 2.count #unique features for each sparse field,and record dense feature field name

fixlen_feature_columns = [SparseFeat(feat, vocabulary_size=data[feat].max() + 1, embedding_dim=4)
                          for i, feat in enumerate(sparse_features)] + [DenseFeat(feat, 1, )
                                                                        for feat in dense_features]

dnn_feature_columns = fixlen_feature_columns
linear_feature_columns = fixlen_feature_columns
feature_names = get_feature_names(linear_feature_columns + dnn_feature_columns)
    
pd.set_option('display.max_columns', len(data))
print("Shape:",data.shape)
data

Loading Time: 31.18088722229004
Shape: (3000000, 34)


,label,I1,I2,I3,I4,I5,I6,I7,I8,I9,I10,I11,I12,I13,C1,C2,C3,C4,C5,C6,C7,C8,C9,C10,C11,C12,C13,C14,C15,C16,C17,C18,C19,C20
0,0,1.0,1,5.0,0.0,1382.0,4.0,15.0,2.0,181.0,1.0,2.0,0.0,2.0,604,268,43,7,10058,79,2,29947,3564,474,4,5888,9,4253,249,3,0,3,69,30715
1,0,2.0,0,44.0,1.0,102.0,8.0,2.0,2.0,4.0,1.0,1.0,0.0,4.0,604,514,43,18,6686,23,2,7715,1540,2629,17,9756,0,3043,249,1,0,3,69,23237
2,0,2.0,0,1.0,14.0,767.0,89.0,4.0,2.0,245.0,1.0,3.0,3.0,45.0,219,25,43,7,9042,23,2,10480,1858,2102,2,4660,6,876,0,0,11,3,0,0
3,0,0.0,893,0.0,0.0,4392.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,604,96,43,18,2155,23,2,42827,4544,656,2,1003,1,1995,0,0,0,3,0,0
4,0,3.0,-1,0.0,0.0,2.0,0.0,3.0,0.0,0.0,1.0,1.0,0.0,0.0,785,381,43,2,7919,23,2,14642,4560,2935,4,6063,1,659,0,0,0,2,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2999995,1,1.0,888,0.0,0.0,503.0,35.0,8.0,1.0,2.0,1.0,3.0,0.0,0.0,527,123,106,7,7062,216,2,8262,3672,2931,9,833,0,792,0,0,0,2,0,0
2999996,1,0.0,-1,0.0,0.0,2556.0,0.0,20.0,0.0,0.0,0.0,1.0,0.0,0.0,30,325,106,9,4974,23,2,4609,1873,904,4,5423,9,730,0,0,10,8,0,0
2999997,1,0.0,-1,0.0,0.0,10.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,30,268,43,18,6818,23,0,10480,2263,2715,17,2182,5,3840,249,3,0,11,69,23797
2999998,0,0.0,6,0.0,0.0,40271.0,0.0,0.0,22.0,10.0,0.0,0.0,0.0,0.0,30,20,43,7,4792,397,2,10480,2093,290,2,2809,9,1812,0,0,0,5,0,0


In [68]:
#DeepFM_model was previously trained on the reduced dataset above
DeepFM_model = load_model('../models/DeepFM-02.h5',custom_objects)

In [35]:
def get_num_range(dense_features):
    #Returns (range) of values for each I1-I13 integer features
    num_data_distrib = []
    for feat in dense_features:
        feature = data[feat]
        #num_data_distrib.append([feature.min(), feature.max(), np.std(feature), feature.mean()])
        #num_data_distrib.append([feature.min(), feature.max(), feature.max() - feature.min()])\
        num_data_distrib.append([feature.max() - feature.min()])
    
    return np.asarray(num_data_distrib)
    
num_data_distrib = get_num_range(dense_features)
num_data_distrib.shape

(13, 1)

In [36]:
def get_cat_range(RecSys, sparse_features):
#Returns (range) of values for each C1-C20 categorical feature embedding

    cat_data_distrib = []
    for category in sparse_features:
        layer = 'sparse_emb_' + category
        weights = RecSys.get_layer(layer).get_weights()[0]
        local_cat_data_distrib = []
        for col in weights.T:
            #local_cat_data_distrib.append([col.min(), col.max(), np.std(col), col.mean()])
            #local_cat_data_distrib.append([col.min(), col.max(), col.max() - col.min()])
            local_cat_data_distrib.append([col.max() - col.min()])
        cat_data_distrib.append(local_cat_data_distrib)

    return np.asarray(cat_data_distrib)

cat_data_distrib = get_cat_range(DeepFM_model, sparse_features)
cat_data_distrib.shape

(20, 4, 1)

In [47]:
def get_feat_ranges(num_data_distrib, cat_data_distrib):
    
    feat_ranges = list(num_data_distrib.copy())
    for feat in cat_data_distrib:
        for dim in feat:
            feat_ranges.append(dim)
    
    return np.asarray(feat_ranges).reshape(1,-1)

feat_ranges = get_feat_ranges(num_data_distrib, cat_data_distrib)
feat_ranges.shape

(1, 93)

Learn Embedding Inversion Function
-

In [123]:
def get_cat_weights(RecSys):
#Returns the mapping from each feature element to their respective embedding vector representations

    cat_weights = []
    for category in sparse_features:
        layer = 'sparse_emb_' + category
        cat_weights.append(RecSys.get_layer(layer).get_weights()[0])
    for feat in cat_weights:
        print(feat.shape)
    return cat_weights

In [125]:
def learn_emb_inversion(RecSys):
#Train model to invert embedding vectors to one-hot encodings representation of categories
    
    cat_weights = get_cat_weights(RecSys)
    for i in range(len(cat_weights)):

        train_indices = np.asarray(range(len(cat_weights[i])))
        train_labels = np.zeros((train_indices.size, train_indices.max()+1))
        train_labels[np.arange(train_indices.size),train_indices] = 1
        print("Size:", sys.getsizeof(train_labels))

        model = tf.keras.Sequential([
            tf.keras.layers.InputLayer(input_shape=(4,)),
            tf.keras.layers.Dense(128, activation='relu'),
            tf.keras.layers.Dense(256, activation='relu'),
            tf.keras.layers.Dense(256, activation='relu'),
            tf.keras.layers.Dense(256, activation='relu'),
            tf.keras.layers.Dense(256, activation='relu'),
            tf.keras.layers.Dense(len(train_labels), activation='sigmoid')
        ]) #outputs integer values for single categorical feature based on embedding vec, to be given to DeepFM

        model.compile(
            loss=tf.keras.losses.binary_crossentropy,
            optimizer=tf.keras.optimizers.Adam(lr=0.00001),
            metrics=[
                tf.keras.metrics.BinaryAccuracy(name='accuracy'),
                tf.keras.metrics.Precision(name='precision'),
                tf.keras.metrics.Recall(name='recall')
            ]
        )

        history = model.fit(cat_weights[i], train_labels, epochs=100)
        model_name = '../models/inverse_emb_models/inverse_emb_' + sparse_features[i] + '.h5'
        save_model(model, model_name)# save inversion model for each categorical column
        
#learn_emb_inversion(DeepFM_model)

(1391, 4)
(544, 4)
(290, 4)
(19, 4)
(11631, 4)
(606, 4)
(3, 4)
(45638, 4)
(5057, 4)
(3108, 4)
(26, 4)
(10845, 4)
(10, 4)
(4461, 4)
(2007, 4)
(4, 4)
(17, 4)
(15, 4)
(83, 4)
(52069, 4)
Size: 15479160
Epoch 1/100
44/44 [==============================] - 1s 7ms/step - loss: 0.6931 - accuracy: 0.7130 - precision: 7.1089e-04 - recall: 0.2828
Epoch 2/100
44/44 [==============================] - 0s 6ms/step - loss: 0.6928 - accuracy: 0.8093 - precision: 7.0002e-04 - recall: 0.1850
Epoch 3/100
44/44 [==============================] - 0s 6ms/step - loss: 0.6923 - accuracy: 0.8209 - precision: 7.2650e-04 - recall: 0.1806
Epoch 4/100
44/44 [==============================] - 0s 6ms/step - loss: 0.6915 - accuracy: 0.8350 - precision: 6.7492e-04 - recall: 0.1542
Epoch 5/100
44/44 [==============================] - 0s 7ms/step - loss: 0.6901 - accuracy: 0.8599 - precision: 7.2907e-04 - recall: 0.1417
Epoch 6/100
44/44 [==============================] - 0s 6ms/step - loss: 0.6876 - accuracy: 0.8922 - p

44/44 [==============================] - 0s 5ms/step - loss: 0.0061 - accuracy: 0.9993 - precision: 0.0000e+00 - recall: 0.0000e+00
Epoch 57/100
44/44 [==============================] - 0s 4ms/step - loss: 0.0061 - accuracy: 0.9993 - precision: 0.0000e+00 - recall: 0.0000e+00
Epoch 58/100
44/44 [==============================] - 0s 5ms/step - loss: 0.0061 - accuracy: 0.9993 - precision: 0.0000e+00 - recall: 0.0000e+00
Epoch 59/100
44/44 [==============================] - 0s 5ms/step - loss: 0.0061 - accuracy: 0.9993 - precision: 0.0000e+00 - recall: 0.0000e+00
Epoch 60/100
44/44 [==============================] - 0s 6ms/step - loss: 0.0061 - accuracy: 0.9993 - precision: 0.0000e+00 - recall: 0.0000e+00
Epoch 61/100
44/44 [==============================] - 0s 6ms/step - loss: 0.0061 - accuracy: 0.9993 - precision: 0.0000e+00 - recall: 0.0000e+00
Epoch 62/100
44/44 [==============================] - 0s 5ms/step - loss: 0.0061 - accuracy: 0.9993 - precision: 0.0000e+00 - recall: 0.0000e+0

17/17 [==============================] - 0s 3ms/step - loss: 0.6859 - accuracy: 0.8396 - precision: 0.0017 - recall: 0.1504
Epoch 14/100
17/17 [==============================] - 0s 4ms/step - loss: 0.6842 - accuracy: 0.8555 - precision: 0.0016 - recall: 0.1271
Epoch 15/100
17/17 [==============================] - 0s 3ms/step - loss: 0.6819 - accuracy: 0.8721 - precision: 0.0021 - recall: 0.1434
Epoch 16/100
17/17 [==============================] - 0s 3ms/step - loss: 0.6793 - accuracy: 0.8885 - precision: 0.0017 - recall: 0.0985
Epoch 17/100
17/17 [==============================] - 0s 3ms/step - loss: 0.6756 - accuracy: 0.9009 - precision: 0.0017 - recall: 0.0879
Epoch 18/100
17/17 [==============================] - 0s 3ms/step - loss: 0.6716 - accuracy: 0.9119 - precision: 0.0019 - recall: 0.0908
Epoch 19/100
17/17 [==============================] - 0s 3ms/step - loss: 0.6664 - accuracy: 0.9206 - precision: 0.0022 - recall: 0.0951
Epoch 20/100
17/17 [==============================] - 

17/17 [==============================] - 0s 4ms/step - loss: 0.0188 - accuracy: 0.9982 - precision: 0.0000e+00 - recall: 0.0000e+00
Epoch 71/100
17/17 [==============================] - 0s 4ms/step - loss: 0.0184 - accuracy: 0.9982 - precision: 0.0000e+00 - recall: 0.0000e+00
Epoch 72/100
17/17 [==============================] - 0s 3ms/step - loss: 0.0183 - accuracy: 0.9982 - precision: 0.0000e+00 - recall: 0.0000e+00
Epoch 73/100
17/17 [==============================] - 0s 3ms/step - loss: 0.0179 - accuracy: 0.9982 - precision: 0.0000e+00 - recall: 0.0000e+00
Epoch 74/100
17/17 [==============================] - 0s 3ms/step - loss: 0.0177 - accuracy: 0.9982 - precision: 0.0000e+00 - recall: 0.0000e+00
Epoch 75/100
17/17 [==============================] - 0s 3ms/step - loss: 0.0173 - accuracy: 0.9982 - precision: 0.0000e+00 - recall: 0.0000e+00
Epoch 76/100
17/17 [==============================] - 0s 3ms/step - loss: 0.0170 - accuracy: 0.9982 - precision: 0.0000e+00 - recall: 0.0000e+0

10/10 [==============================] - 0s 4ms/step - loss: 0.6808 - accuracy: 0.8943 - precision: 0.0040 - recall: 0.1203
Epoch 29/100
10/10 [==============================] - 0s 3ms/step - loss: 0.6791 - accuracy: 0.8980 - precision: 0.0026 - recall: 0.0745
Epoch 30/100
10/10 [==============================] - 0s 3ms/step - loss: 0.6774 - accuracy: 0.9062 - precision: 0.0042 - recall: 0.1107
Epoch 31/100
10/10 [==============================] - 0s 3ms/step - loss: 0.6754 - accuracy: 0.9120 - precision: 0.0036 - recall: 0.0894
Epoch 32/100
10/10 [==============================] - 0s 3ms/step - loss: 0.6730 - accuracy: 0.9150 - precision: 0.0025 - recall: 0.0599
Epoch 33/100
10/10 [==============================] - 0s 3ms/step - loss: 0.6707 - accuracy: 0.9203 - precision: 0.0034 - recall: 0.0757
Epoch 34/100
10/10 [==============================] - 0s 4ms/step - loss: 0.6677 - accuracy: 0.9252 - precision: 0.0043 - recall: 0.0897
Epoch 35/100
10/10 [==============================] - 

...

1/1 [==============================] - 0s 3ms/step - loss: 0.6896 - accuracy: 0.7289 - precision: 0.0893 - recall: 0.3333
Epoch 58/100
1/1 [==============================] - 0s 3ms/step - loss: 0.6895 - accuracy: 0.7289 - precision: 0.0893 - recall: 0.3333
Epoch 59/100
1/1 [==============================] - 0s 3ms/step - loss: 0.6894 - accuracy: 0.7467 - precision: 0.0962 - recall: 0.3333
Epoch 60/100
1/1 [==============================] - 0s 3ms/step - loss: 0.6893 - accuracy: 0.7511 - precision: 0.0980 - recall: 0.3333
Epoch 61/100
1/1 [==============================] - 0s 4ms/step - loss: 0.6892 - accuracy: 0.7556 - precision: 0.1000 - recall: 0.3333
Epoch 62/100
1/1 [==============================] - 0s 3ms/step - loss: 0.6892 - accuracy: 0.7644 - precision: 0.1042 - recall: 0.3333
Epoch 63/100
1/1 [==============================] - 0s 3ms/step - loss: 0.6891 - accuracy: 0.7644 - precision: 0.1042 - recall: 0.3333
Epoch 64/100
1/1 [==============================] - 0s 3ms/step - lo

3/3 [==============================] - 0s 4ms/step - loss: 0.6919 - accuracy: 0.7987 - precision: 0.0099 - recall: 0.1586
Epoch 18/100
3/3 [==============================] - 0s 3ms/step - loss: 0.6918 - accuracy: 0.8013 - precision: 0.0105 - recall: 0.1664
Epoch 19/100
3/3 [==============================] - 0s 3ms/step - loss: 0.6917 - accuracy: 0.8078 - precision: 0.0104 - recall: 0.1586
Epoch 20/100
3/3 [==============================] - 0s 3ms/step - loss: 0.6916 - accuracy: 0.8103 - precision: 0.0115 - recall: 0.1742
Epoch 21/100
3/3 [==============================] - 0s 3ms/step - loss: 0.6915 - accuracy: 0.8164 - precision: 0.0113 - recall: 0.1625
Epoch 22/100
3/3 [==============================] - 0s 3ms/step - loss: 0.6914 - accuracy: 0.8231 - precision: 0.0119 - recall: 0.1664
Epoch 23/100
3/3 [==============================] - 0s 4ms/step - loss: 0.6913 - accuracy: 0.8286 - precision: 0.0110 - recall: 0.1468
Epoch 24/100
3/3 [==============================] - 0s 3ms/step - lo

3/3 [==============================] - 0s 4ms/step - loss: 0.6787 - accuracy: 0.9179 - precision: 0.0126 - recall: 0.0752
Epoch 78/100
3/3 [==============================] - 0s 3ms/step - loss: 0.6782 - accuracy: 0.9180 - precision: 0.0126 - recall: 0.0752
Epoch 79/100
3/3 [==============================] - 0s 3ms/step - loss: 0.6778 - accuracy: 0.9194 - precision: 0.0109 - recall: 0.0635
Epoch 80/100
3/3 [==============================] - 0s 5ms/step - loss: 0.6771 - accuracy: 0.9189 - precision: 0.0102 - recall: 0.0596
Epoch 81/100
3/3 [==============================] - 0s 3ms/step - loss: 0.6765 - accuracy: 0.9207 - precision: 0.0111 - recall: 0.0635
Epoch 82/100
3/3 [==============================] - 0s 7ms/step - loss: 0.6764 - accuracy: 0.9210 - precision: 0.0158 - recall: 0.0908
Epoch 83/100
3/3 [==============================] - 0s 5ms/step - loss: 0.6756 - accuracy: 0.9222 - precision: 0.0141 - recall: 0.0791
Epoch 84/100
3/3 [==============================] - 0s 5ms/step - lo

1628/1628 [==============================] - 219s 135ms/step - loss: 2.2839e-04 - accuracy: 1.0000 - precision: 0.0000e+00 - recall: 0.0000e+00
Epoch 33/100
1628/1628 [==============================] - 221s 136ms/step - loss: 2.2838e-04 - accuracy: 1.0000 - precision: 0.0000e+00 - recall: 0.0000e+00
Epoch 34/100
1628/1628 [==============================] - 219s 135ms/step - loss: 2.2837e-04 - accuracy: 1.0000 - precision: 0.0000e+00 - recall: 0.0000e+00
Epoch 35/100
1628/1628 [==============================] - 212s 130ms/step - loss: 2.2837e-04 - accuracy: 1.0000 - precision: 0.0000e+00 - recall: 0.0000e+00
Epoch 36/100
1628/1628 [==============================] - 205s 126ms/step - loss: 2.2836e-04 - accuracy: 1.0000 - precision: 0.0000e+00 - recall: 0.0000e+00
Epoch 37/100
1628/1628 [==============================] - 211s 130ms/step - loss: 2.2836e-04 - accuracy: 1.0000 - precision: 0.0000e+00 - recall: 0.0000e+00
Epoch 38/100
1628/1628 [==============================] - 198s 121ms/st

1628/1628 [==============================] - 211s 129ms/step - loss: 2.2825e-04 - accuracy: 1.0000 - precision: 0.0000e+00 - recall: 0.0000e+00
Epoch 85/100
1628/1628 [==============================] - 239s 147ms/step - loss: 2.2825e-04 - accuracy: 1.0000 - precision: 0.0000e+00 - recall: 0.0000e+00
Epoch 86/100
1628/1628 [==============================] - 235s 145ms/step - loss: 2.2825e-04 - accuracy: 1.0000 - precision: 0.0000e+00 - recall: 0.0000e+00
Epoch 87/100
1628/1628 [==============================] - 217s 133ms/step - loss: 2.2825e-04 - accuracy: 1.0000 - precision: 0.0000e+00 - recall: 0.0000e+00
Epoch 88/100
1628/1628 [==============================] - 233s 143ms/step - loss: 2.2825e-04 - accuracy: 1.0000 - precision: 0.0000e+00 - recall: 0.0000e+00
Epoch 89/100
1628/1628 [==============================] - 242s 149ms/step - loss: 2.2825e-04 - accuracy: 1.0000 - precision: 0.0000e+00 - recall: 0.0000e+00
Epoch 90/100
1628/1628 [==============================] - 220s 135ms/st

In [119]:
#Convert categorical feature values to embedding vectors, and concatenate as feature values
def cat_to_emb(node):
    
    emb_node = list(node[:len(dense_features)])
    for i in range(len(emb_node)):
        if int(emb_node[i]) == -1:
            emb_node[i] = None

    for i in range(len(source_node[len(dense_features):])):
        #Missing feature value, mark embedding vector to 
        if int(node[i+len(dense_features)]) == 0:
            for _ in range(4):
                emb_node.append(None)
        else:
            emb = cat_weights[i][int(node[i+len(dense_features)])]
            for val in emb:
                emb_node.append(val)

    return np.asarray(emb_node)
        
        
def emb_to_cat(emb_node):
#Project categorical embeddings back to integer (one-hot encoding) to be queried by DeepFM
    
    #Not necessary for querying source node
    if len(emb_node) == len(sparse_features) + len(dense_features):
        return emb_node
    
    cat_embeddings = emb_node
    

def query_RecSys(RecSys, node):
    
    transf_numeric_feats = mms.transform(np.asarray(node[:len(dense_features)]).reshape(1, -1))
    transf_node = [*transf_numeric_feats[0], *node[len(dense_features):]]
    feature_names = dense_features + sparse_features
    queryrec_input = {feature_names[name]:np.asarray([transf_node[name]]) for name in range(len(feature_names))}
    rec = RecSys.predict(queryrec_input)
    return rec.round() #returns model recommendation

#First row of training data, removed rec label: 0, 39 features
source_node = np.asarray(data.iloc[5])[1:]
rec = query_RecSys(DeepFM_model, source_node)
print(rec)

[[1.]]


In [120]:
pos_sum = 0
for i in range(100):
    index = 1000+i
    source_node = np.asarray(data.iloc[index])[1:]
    rec = query_RecSys(DeepFM_model, source_node)
    if rec == 1:
        pos_sum +=1
        
pos_sum

12

In [18]:
class Node:
    def __init__(self, position, f_cost):
        
        #Position list([row, column])
        self.position = list(position) # [n1, n2, ..., n|N|]
        self.f_cost = f_cost #Distance for evaluation function to compare nodes in priority queue
        
  # Nodes with the same f_cost
    def __eq__(self, other):
        return isinstance(other, Node) and self.f_cost == other.f_cost
  
    # Node is less than other node
    def __lt__(self, other):
        return isinstance(other, Node) and self.f_cost < other.f_cost

    # Node is greater than other node
    def __gt__(self, other):
        return isinstance(other, Node) and self.f_cost > other.f_cost

In [7]:
def get_distance(source_node, query_node, feat_scaler, distance='Manhattan'):
    #returns distance for ndim feature space
    
    dif = query_node - source_node
    scaled_dif = dif / feat_scaler
    
    # More efficient at scale
    if distance == 'Manhattan':
        return sum(abs(scaled_dif))
    
    if distance == 'Euclidean':
        dsum = 0
        for feat in dif:
            dsum += feat**2
        return math.sqrt(dsum)


def generate_precursor_solutions(source_node, feat_ranges, RecSys, k=20, nsteps=6, plot_search=False):

    step_sizes = (feat_ranges/nsteps) #How large of a step to take for each respective int feature
    feat_scaler = 1/np.asarray(feat_ranges) #Scale to normalize feature step size when calculating distance
    
    feat_steps = []
    for feat in range(len(feat_ranges)):
        feat_steps.append(np.arange(step_sizes[feat],feat_ranges[feat],step_sizes[feat]))
    feat_steps = np.asarray(feat_steps)
    
    source_rec = query_RecSys(RecSys, source_node)
    
    if plot_search:
        plt.plot(source_feats[0],source_feats[1],'r+')

    discovered = {}

    while len(discovered) < k:
        unit_vector = generate_unit_vectors(len(source_node),1)[0]
        for step in range(nsteps-1):
            scaled_vector = np.asarray(unit_vector * feat_steps.T[step] + source_node)
            query_rec = query_RecSys(RecSys, scaled_vector)
            
            if plot_search:
                if query_rec == source_rec:
                    plt.plot(scaled_vector[0],scaled_vector[1],'bo')
                else:
                    plt.plot(scaled_vector[0],scaled_vector[1],'y^')
            
            if source_rec != query_rec:
                # distance heuristic
                discovered[tuple(scaled_vector)] = get_distance(source_node, scaled_vector, feat_scaler)
                break
    
    return discovered
                
#source_node = np.asarray([0,0])
#feat_ranges = np.asarray([10,10])
#prelim_discovered = generate_precursor_solutions(source_node, feat_ranges, plot_search=True, nsteps=10)

In [14]:
#TODO: Adapative gradient to make largest jumps in k-closest solutions in beginning, then lower over time

def find_adjs(node, feat_scaler, learning_rate):
    #Returns all nodes adjacent to current solution in ndim feature space
    
    adj_nodes = []
    for dim in range(len(node)):
        #Check if dim is a missing value, as marked in cat_dims_to_emb()
        if node[dim] != None
            temp = node.copy()
            temp[dim] = temp[dim] + learning_rate / feat_scaler[dim]
            adj_nodes.append(temp)
            temp = node.copy()
            temp[dim] = temp[dim] - learning_rate / feat_scaler[dim]
            adj_nodes.append(temp)
    return adj_nodes


def a_star_search(source_node, prelim_discovered, feat_ranges, RecSys, k=100, learning_rate=0.25):
    
    #Analytics to inform how to handle degree of branching
    nodes_generated = 0
    nodes_visited = 0
    
    frontier = [] # Priority Queue
    discovered = [] #Vector of k-closest feature interactions to opposite recommendation
    visited = {} # True corresponds to visited for low complexity "visited" search
    
    #Add solutions from uniformly distributed vectors to priority queue (frontier)
    for position in prelim_discovered:
        cur = Node(position, prelim_discovered[position]) #(Node.position, Node.f_cost)
        heapq.heappush(frontier, cur)
        nodes_generated += 1
        
    #Scale to normalize learning rate for each feature
    feat_scaler = 1/np.asarray(feat_ranges)
    
    source_rec = query_RecSys(RecSys, source_node) #rec value to be explained
     
    # loop till queue is empty, or k solutions were discovered
    #TODO: Want to define stop criteria based on no solutions approaching center
    while frontier and len(discovered) < k:
 
        # dequeue front node
        query_node = heapq.heappop(frontier)
        #print(node, end=' ')
        nodes_visited += 1
        
        #Check goal state
        query_rec = query_RecSys(RecSys, np.asarray(query_node.position))
        if query_rec != source_rec:
            discovered.append(query_node)
            heapq.heappush(discovered, query_node) #priority queue so discovered is sorted by feature importance
            #print("soln:",query_node.position)
 
        #TODO: Want to add these to priority queue before visiting them
        adjs = find_adjs(query_node.position, feat_ranges, learning_rate) #every node adjacent to the current node
        for adj_node in adjs:
            
            in_bfet = True
            for i in range(len(adj_node)):
                if adj_node[i] >= feat_ranges[i] or adj_node[i] <= -1*feat_ranges[i]:
                    in_bfet = False
            
            if in_bfet and tuple(adj_node) not in visited.keys():
                
                cur_adj = Node(adj_node, get_distance(source_node, adj_node, feat_scaler))
                heapq.heappush(frontier, cur_adj)
                visited[tuple(adj_node)] = True
                nodes_generated += 1
    
    print("Nodes Visited:",nodes_visited)
    print("Nodes Generated:",nodes_generated)
    print("Visited Length:",len(visited))

    return discovered

7

TODO:
-
- Modify find_adjs() to only generate adjacents in direction of source node, to reduce memory complexity of frontier priority queue
- Modify find_adjs() to take certain step size based on learning rate
- Generate Ground Truth (GT) evaluation model with manufactured decision boundary surrounding source node, to provide a GT of explainability as defined by the function that creates the search space. This function will be used to compare the accuracy of GD-FETS to LIME
- Use 2 Std devs of each feature of training data as feat_range for data distribution to ignore outliers that would cause certain features to be traversed with too large a step size
- Implement solutions clustering method for identifying which multi-way feature interaction a sample is expressing
- Implement method to display the magnitude of instability for top-k feature interactions
- T-tests to demonstrate signifiance of GD-FETS identifying the same feature interactions as expreesed in the GT explanation
- Implement LIME/SHAP to give comparison of their performance using a Ground Truth decision boundary, to that of GD-FETS
- Implement one-way feature-interaction for GD-FETS for direct compariosn to LIME
- Describe evaluation methodology in manuscript, and describe results
- Would like to do: Use T-SNE to project categorical embedding vectors from 4 dimension to 2 (reduce complexity of search space), can also collapse the dimensionality to a single dimension, although => information loss
- Use generators to dynamically create one-hot encoding labels for training embedding inversion models, for features with very large dictionary sizes, due to memory constraints
- Future Direction: Implement reinforcement learning agent to repalce the A* search engine, as RL policy can learn the most common explanability for a given model and feature values, which can be leveraged fro search.